## Convert the (H,W,C) mask to (H,W,K)

In [1]:
import numpy as np

In [2]:
palette = [[0], [252], [253],[254], [255]]

In [3]:
import os
import imageio

In [4]:
mask = imageio.v2.imread(r'D:\Segthordataset\train\Patient_02_png\GT.nii.gz\Patient0_02141.png')

In [5]:
mask.shape

(512, 512)

In [6]:
mask = np.expand_dims(mask, axis=2)

In [7]:
mask.shape

(512, 512, 1)

In [8]:
np.unique(mask, return_counts=True)

(Array([  0, 252, 254, 255], dtype=uint8),
 array([250763,    568,  10638,    175], dtype=int64))

In [9]:
def mask_to_onehot(mask, palette):
    semantic_map = []
    for color in palette:
        equality = np.equal(mask,color)
        class_map = np.all(equality,axis=-1)
        semantic_map.append(class_map)
    semantic_map = np.stack(semantic_map, axis=-1).astype(np.float32)
    return semantic_map

In [10]:
mask_ = mask_to_onehot(mask, palette)
mask_.shape

(512, 512, 5)

In [12]:
np.unique(mask_[:,:,2], return_counts=True)

(array([0.], dtype=float32), array([262144], dtype=int64))

In [19]:
def onehot_to_mask(mask, palette):
    """
    Converts a mask (H, W, K) to (H, W, C)
    """
    x = np.argmax(mask, axis=-1)
    colour_codes = np.array(palette)
    x = np.uint8(colour_codes[x.astype(np.uint8)])
    return x

In [20]:
pred = onehot_to_mask(mask, palette)

In [21]:
pred.shape

(512, 512, 1)

In [13]:
import torch

In [45]:
test_mask = np.ones([1,5,5,5])

In [46]:
test_mask

array([[[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]]]])

In [47]:
test_mask.shape

(1, 5, 5, 5)

In [48]:
test_pred = onehot_to_mask(test_mask, palette)

In [49]:
test_pred.shape

(1, 5, 5, 1)

In [50]:
test_pred

array([[[[0],
         [0],
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         [0],
         [0]]]], dtype=uint8)

In [14]:
gt = torch.Tensor([[
        [[0, 1, 1, 0],
         [1, 0, 0, 1],
         [1, 0, 0, 1],
         [0, 1, 0.7, 0]],
        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0.5, 1, 0],
         [0, 0, 0, 0]],
        [[1, 0, 0, 1],
         [0, 1, 0.2, 0],
         [0, 0, 0, 0],
         [1, 0, 0, 1]]],
        [
            [[0, 1, 1, 0],
             [1, 0, 0, 1],
             [1, 0, 0, 1],
             [0, 1, 1, 0]],
            [[0, 0, 0, 0],
             [0, 0, 0, 0],
             [0, 1, 1, 0],
             [0, 0, 0, 0]],
            [[1, 0, 0, 1],
             [0, 1, 1, 0],
             [0, 0, 0, 0],
             [1, 0, 0, 1]]]
    ])

In [15]:
gt.shape

torch.Size([2, 3, 4, 4])

In [16]:
gt[gt>0.5] = 1
gt[gt<=0.5] = 0

In [17]:
gt

tensor([[[[0., 1., 1., 0.],
          [1., 0., 0., 1.],
          [1., 0., 0., 1.],
          [0., 1., 1., 0.]],

         [[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 1., 0.],
          [0., 0., 0., 0.]],

         [[1., 0., 0., 1.],
          [0., 1., 0., 0.],
          [0., 0., 0., 0.],
          [1., 0., 0., 1.]]],


        [[[0., 1., 1., 0.],
          [1., 0., 0., 1.],
          [1., 0., 0., 1.],
          [0., 1., 1., 0.]],

         [[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 1., 1., 0.],
          [0., 0., 0., 0.]],

         [[1., 0., 0., 1.],
          [0., 1., 1., 0.],
          [0., 0., 0., 0.],
          [1., 0., 0., 1.]]]])